In [ ]:
# written by Maximilian Ullherr, maximilian.ullherr@physik.uni-wuerzburg.de,
# Lehrstuhl fuer Roentgenmikroskopie/Universitaet Wuerzburg, Josef-Martin-Weg 63, 97074 Wuerzburg, Germany
# last updated 2019-04-26

%pylab
import os
set_printoptions(precision=3)
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import interp1d
os.makedirs('fig', exist_ok=True)

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
colors =   ['#1f77b4', '#ff7f0e', '#2ca02c', '#d64748', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
colors_B = ['#4c702c', '#a61718', '#1f97a4', '#af700e', '#b497bd', '#0c764b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

print(colors)

# SNR simulations

## aim

compute the SNR for different energy weighting schemes, the 

theoretically ideal w = signal strength and the 

practically useful w = E < E_lim

counting detector w = const

energy integrating detector (scintillator) w = E

## computation

multiplying a weight after data aquisition (digitally) affects signal intensity linearly and noise power squarely

excluding detection events via energy limits affects both signal intensity and noise power linearly, but 0^2 = 0 and 1^2=1 so it makes no difference


In [ ]:
try:
    # physical data must be energy [eV] in the first column and absorption constant [1/cm] in the second
    mu_Al = genfromtxt("physical_data/Al_mu_cm.txt", unpack=True, skip_header=1)
    mu_Fe = genfromtxt("physical_data/Fe_mu_cm.txt", unpack=True, skip_header=1)
    print('physical data loaded from disk')
    saving_enabled = True
except OSError:
    print('WARNING: no actual physical data found, using E^-3 estimate')
    E = logspace(3, 5, 5000)
    # constants were generated by fitting a E^-3 model to physical data over the range of 2 to 60 keV,
    # see cell below (inaccurate for absorption edges and when compton scattering becomes relevant)
    mu_Al = E, 82768440112531.06*E**-3
    mu_Fe = E, 1328394990000272.8*E**-3
    saving_enabled = False

E_tube = 30
E = linspace(1, E_tube, 1000)

plt.clf()
plt.loglog(*mu_Al)
plt.loglog(*mu_Fe)
plt.grid()
plt.ylabel('absorption [1/cm]')
plt.xlabel('energy [eV]')
plt.legend(['Al', 'Fe'])
plt.xlim(1e3, 2e5)
display(plt.gcf())

def sample_transmission(E, thickness):
    # thickness is is mm
    mu = interp1d(*mu_Al, fill_value='extrapolate')(E*1000)
    T_sample = exp(-mu*thickness/10)
    return T_sample

def sample_transmission_Fe(E, thickness):
    # thickness is is mm
    mu = interp1d(*mu_Fe, fill_value='extrapolate')(E*1000)
    T_sample = exp(-mu*thickness/10)
    return T_sample

def signal_strength_Fe_Al(E):
    mu1 = interp1d(*mu_Fe, fill_value='extrapolate')(E*1000)
    mu2 = interp1d(*mu_Al, fill_value='extrapolate')(E*1000)
    return abs(mu1-mu2)
def signal_strength_Fe(E):
    return interp1d(*mu_Fe, fill_value='extrapolate')(E*1000)
def signal_strength_Al(E):
    return interp1d(*mu_Al, fill_value='extrapolate')(E*1000)

def signal_strength_phase(E):
    return E**-2

plt.clf()
plt.plot(E, sample_transmission(E, 0.1))
plt.plot(E, sample_transmission(E, 1))
plt.grid()
plt.ylabel('transmission')
plt.xlabel('energy [keV]')
plt.legend(['100 $\mu$m Al', '1 mm Al'])
display(plt.gcf())
plt.close('all')

In [ ]:
plt.clf()
loc = logical_and(mu_Al[0]>2e3, mu_Al[0]<6e4)
for mu, name in zip((mu_Al, mu_Fe), ('Al', 'Fe')):
    E_fit = mu[0][loc]
    mu_fit = mu[1][loc]

    from scipy.optimize import curve_fit

    def model(E, a):
        return log(a*E**-3)

    a, a_var = curve_fit(model, E_fit, log(mu_fit))

    print(a[0])
    plt.loglog(E_fit/1e3, mu_fit, label=name)
    plt.loglog(E_fit/1e3, exp(model(E_fit, a)), ls='--', label=name+' fit')
plt.grid(); plt.legend()
plt.ylabel('absorption'); plt.xlabel('energy [keV]')
display(plt.gcf())
plt.close('all')


In [ ]:
E = linspace(5, 60, 1000)
fig, ax = plt.subplots(1, 1, figsize=(6, 3))

ideal_CEW = signal_strength_Al(E)
ideal_CEW /= ideal_CEW.mean()
ax.plot(E, ideal_CEW, label='ideal CEW ($w$=SDS)')
ax.plot(E, ones_like(E), ls=':', label='no weighting')
ax.plot(E, E/E.mean(), ls='--', label='energy integrating DEW')
mask = (E<18).astype('f4')
mask /= mask.mean()
ax.plot(E, mask, ls='--', label='threshold DEW weight')
ax.set_ylim(0, 10)

ax.grid()
ax.set_ylabel('weighting factor ($w$ or $I$)')
ax.set_xlabel('energy [keV]')
ax.legend()
if saving_enabled:
    fig.savefig('fig/weighting_factor_energy.pdf')
display(fig)
plt.close(fig)

In [ ]:
print('warning: energy weighted SNR assumes efficient conversion (c**2 term >> c term)')

class SNR_Simulation:
    def __init__(self, bin_width=0.01, verbose=0):
        self.log = ''
        self.bin_width = bin_width # keV
        self.min_energy = 2  # keV
        self.source_behaviour = None
        self.E_tube = None

        self.source_spectrum = None
        self.verbose = verbose
        self.plot_param = 1.
        
    def init_energy_properties(self, nbins):
        self.detector_abs_spectrum = np.ones(nbins, 'f8')
        self.filter_transparency = np.ones(nbins, 'f8')
        self.sample_transparency = np.ones(nbins, 'f8')
        self.signal = np.zeros(nbins, 'f8')
        
    def set_source(self, E, source_spectrum):
        self.E = E
        self.init_energy_properties(len(self.E))
        self.source_spectrum = source_spectrum
        self.source_intensity = source_spectrum.sum()

    def set_source_bremsspektrum(self, E_tube, limit='power'):
        self._print(f'setting tube voltage to {E_tube} with limit "{limit}"')
        self.E_tube = E_tube

        self.E = arange(self.min_energy//self.bin_width, E_tube//self.bin_width)*self.bin_width #linspace(self.min_energy, E_tube, self.nbins)
        self.init_energy_properties(len(self.E))
        source_spectrum = (E_tube - self.E)
        if limit == 'power':
            source_spectrum /= E_tube
        elif limit == 'current':
            pass
        else:
            raise ValueError('limiting factor for the tube power must either be "power" or "current"')

        self.source_spectrum = source_spectrum
        self.source_intensity = source_spectrum.sum()

    def add_emission_line(self, emission_line_strength, emission_line_energy):
        assert self.E_tube is not None, 'emission lines can only be added to a bremsspektrum'
        if emission_line_strength > 0.0:
            emission_line_strength_real = emission_line_strength * (self.E_tube - emission_line_energy) ** 1.5
            if self.source_behaviour == 'constant_power':
                emission_line_strength_real /= self.E_tube

            k = argmin(abs(self.E - emission_line_energy))
            line_emission_fraction = emission_line_strength_real * self.sample_transparency[k] / (
                        (self.source_spectrum * self.sample_transparency).sum() + emission_line_strength_real *
                        self.sample_transparency[k])
            self._print(f'added emission line with {line_emission_fraction:.1%} intensity at {E[k]:.2f} keV')
            self.source_spectrum[k] += emission_line_strength_real
  
    def set_detector(self, detector_QE_spectrum):
        self.detector_abs_spectrum = detector_QE_spectrum
        
    def set_photoabs_detector(self, abstract_thickness):
        self.detector_abs_spectrum = 1. - exp(-self.E**(-3)*abstract_thickness)
        
    def add_filter(self, thickness):
        # only Al supported, thickness in mm
        self.filter_transparency *= sample_transmission(self.E, thickness)

    def add_absorbing_signal(self, thickness, phase_only=False):
        # only Al supported, thickness in mm
        if not phase_only:
            self.signal += signal_strength_Al(self.E)
        self.sample_transparency *= sample_transmission(self.E, thickness)

    def add_phase_signal(self, signal_strength):
        self.signal += signal_strength * self.E ** -2

    def find_optimal_energy_cutoff(self, E, source_spectrum, sigal_strength, lower_cutoff):
        SNR_lim_weight = zeros_like(E)
        for k, E_lim in enumerate(E):
            lim_weight = (lower_cutoff < E) & (E < E_lim)
            SNR_lim_weight[k] = sum(sigal_strength * source_spectrum * lim_weight) ** 2 / sum(
                source_spectrum * lim_weight ** 2)

        SNR_lim_weight[~isfinite(SNR_lim_weight)] = 0.
        k_opt = argmax(SNR_lim_weight)
        return E[k_opt]

    def find_optimal_monochromatic_SNR(self):
        source_intensity = (self.source_spectrum * self.filter_transparency).sum()
        mono_SNR = self.sample_transparency * self.detector_abs_spectrum * source_intensity * self.signal ** 2
        k = argmax(mono_SNR)
        return self.E[k], mono_SNR[k],  k

    def _print(self, *args):
        for arg in args:
            self.log += str(arg) + ' '
        self.log += '\n'
        
    def compute_SNR_CEW(self, weight=1.):
        return sum(self.signal * self.detected_spectrum * weight) ** 2 / sum(self.detected_spectrum * weight ** 2)
    
    def compute_SNR_DEW(self, weight=1.):
        return sum(self.signal * self.detected_spectrum * weight) ** 2 / sum(self.detected_spectrum * weight)
    
    def compute_transparency(self, weight=1.):
        return sum(self.detected_spectrum * weight) / sum(self.spectrum_without_sample * weight)  
    
    def compute_all_transparency(self, weight=1.):
        self.DA_mono = self.sample_transparency[self.mono_index]
        self.T_noweight = self.compute_transparency()
        self.T_CEW = self.compute_transparency(self.signal)
        self.T_EI = self.compute_transparency(self.E)
        self.T_DEW = self.compute_transparency(self.lim_weight) 
        
    def compute_detector_abs(self, weight=1.):
        weighted = self.detected_spectrum*weight
        return weighted.sum()/(weighted/self.detector_abs_spectrum).sum()
        
    def compute_all_detector_abs(self):
        self.DA_mono = self.detector_abs_spectrum[self.mono_index]
        self.DA_noweight = self.compute_detector_abs()
        self.DA_CEW = self.compute_detector_abs(self.signal)
        self.DA_EI = self.compute_detector_abs(self.E)
        self.DA_DEW = self.compute_detector_abs(self.lim_weight) 
        
    def simulate(self, which='td'):
        self.spectrum_without_sample = self.source_spectrum * self.filter_transparency * self.detector_abs_spectrum
        self.detected_spectrum = self.spectrum_without_sample * self.sample_transparency
        self.source_intensity = self.detected_spectrum.sum()
        #self._print(f'source intensity: {self.detected_spectrum.sum():.4g} detected')
        intensity_scale_k = argmax(self.detected_spectrum)

        self.SNR_noweight = self.compute_SNR_CEW()
        self.SNR_EI_weight = self.compute_SNR_CEW(self.E)
        # ^^^ assumes that c**2 term dominates the sum ^^^
        self.E_DEW_opt_thr = self.find_optimal_energy_cutoff(self.E, self.detected_spectrum, self.signal, self.min_energy)
        self.lim_weight = (self.E < self.E_DEW_opt_thr).astype('f4')
        self.SNR_lim_weight = self.compute_SNR_DEW(self.lim_weight)
        self.SNR_opt_weight = self.compute_SNR_CEW(self.signal)
        
        self.E_mono, self.SNR_mono, self.mono_index = self.find_optimal_monochromatic_SNR()
        
        if 't' in which:
            self.compute_all_transparency()
        if 'd' in which:
            self.compute_all_detector_abs()

    def plot(self):
        intensity_scale_k = argmax(self.source_spectrum)
        self.fig, ax = plt.subplot(1, 1, figsize=(8, 8))
        ax.plot(self.E, self.source_spectrum)
        ax.plot(self.E, self.min_energy * intensity_scale_k / self.lim_weight.max())
        ax.fill_between(E, self.source_spectrum * self.lim_weight, alpha=0.2)
        ax.vlines(self.E_mono, 0, intensity_scale_k * 1.2)
        ax.set_ylim(0, intensity_scale_k * 1.2)
        ax.grid()
        ax.set_ylabel('intensity')
        ax.set_xlabel('energy [keV]')
        ax.legend(['detected spectrum', 'detector bandpass', 'bandpass detected spectrum', 'monochromatic optimum'])

    
from scipy.ndimage import median_filter
def plot_comparison(results, relative_SNR=False, save_path=None, figsize='auto', plot_transparency=True, 
                    plot_sources=True, spectra_mask=None, markers=('1', '2', '2', '1', '3'), ls='', 
                    draw_SNR_maxima=False, relative_yscale=True, xlabel='tube voltage [kV]', xlog=False):
    plot_params = [result.plot_param for result in results]
    if spectra_mask is None:
        spectra_mask = ones(len(results), '?')
    if figsize == 'auto':
        figsize = (8, (5+3*plot_transparency+3*plot_sources))
        
    fig = plt.figure(figsize=figsize)
    naxes = 1+plot_transparency+plot_sources
    gs = fig.add_gridspec(naxes, 1, height_ratios=(5, 3, 3)[:naxes])
    axes = []
    for k in range(naxes):
        axes.append(fig.add_subplot(gs[k]))#, sharex=(axes[0] if k > 0 else None)))
    
    if relative_SNR:
        norm = array([r.SNR_mono for r in results])
    else:
        norm = 1.
        
    SNR_noweight = array([r.SNR_noweight for r in results])/norm
    SNR_EI_weight = array([r.SNR_EI_weight for r in results])/norm
    SNR_opt_weight = array([r.SNR_opt_weight for r in results])/norm
    SNR_lim_weight = array([r.SNR_lim_weight for r in results])/norm
    SNR_mono = array([r.SNR_mono for r in results])/norm
    all_SNR = hstack((SNR_noweight, SNR_EI_weight, SNR_opt_weight, SNR_lim_weight, SNR_mono))
    dynamic_range = all_SNR.max()/all_SNR.min()
    
    plot_kwargs = {'ls': ls, 'markersize':10}
    labels = ('monochrom.', 'optimal CEW', 'optimal DEW', 'no weight', '$\propto$E CEW')
    data = SNR_mono, SNR_opt_weight, SNR_lim_weight, SNR_noweight, SNR_EI_weight
    
    ylim = 1.1*amax(all_SNR)
    if relative_yscale:
        norm = all_SNR.max()
        ylim /= norm
        
    axes[0].plot(plot_params, data[0]/norm, marker=markers[0], label=labels[0], color=colors[0], 
                 markersize=plot_kwargs['markersize'], ls=('--' if plot_kwargs['ls'] == '-' else plot_kwargs['ls']))
    for k in arange(1, 5):
        axes[0].plot(plot_params, data[k]/norm, marker=markers[k], label=labels[k], color=colors[k], **plot_kwargs)
    if draw_SNR_maxima:
        maxima = array([argmax(d) for d in data])
        for k in arange(3, 5):
            if maxima[k] < len(plot_params)-1:
                axes[0].vlines(plot_params[maxima[k]], 0, ylim, color=colors[k%len(colors)], linestyles=':', 
                               label='SNR maxima' if k == 3 else None)
            else:
                print('ignored SNR maxima vline for', labels[k])
    axes[0].grid()
    axes[0].set_yscale('log' if dynamic_range > 20 else 'linear')
    if xlog:
        axes[0].set_xscale('log')
    axes[0].legend(loc='upper right')
    axes[0].set_ylabel('SNR [au]')
    axes[0].set_xlabel(xlabel)
    axes[0].set_ylim(0., ylim)
    
    if plot_sources:
        ylim = 1.1*amax(median_filter(hstack([r.detected_spectrum for r in results]), 3))
        k_c = 0
        
        if relative_yscale:
            norm = ylim/1.1
            ylim = 1.1
        else:
            norm = 1.
        
        for k, r in enumerate(results):
            if spectra_mask[k]:
                if k_c == 0:
                    label1, label2 = f'detected spectra', f'DEW thresholds'
                else:
                    label1, label2 = None, None
                axes[1].plot(r.E, r.detected_spectrum/norm, color=colors_B[k_c%len(colors_B)], label=label1)
                axes[1].vlines(r.E_DEW_opt_thr, 0, ylim, color=colors_B[k_c%len(colors_B)], linestyles=':', label=label2)
                k_c += 1
    
        axes[1].vlines(r.E_mono, 0, ylim, color='black', label=f'monochrom. energy', linestyles='--')
        axes[1].grid()
        axes[1].legend(loc='upper right')
        axes[1].set_ylabel('intensity [au]')
        axes[1].set_xlabel('energy [keV]')
        axes[1].set_ylim(0., ylim)
    
    if plot_transparency:
        ax_index = plot_sources+plot_transparency
        data = array([r.T_mono for r in results]), array([r.T_CEW for r in results]), array([r.T_DEW for r in results]), array([r.T_noweight for r in results]), array([r.T_EI for r in results])
        
        axes[ax_index].plot(plot_params, data[0], label=labels[0], color=colors[0], 
                     ls=('--' if plot_kwargs['ls'] == '-' else plot_kwargs['ls']))
        for k in arange(1, 5):
            axes[ax_index].plot(plot_params, data[k], label=labels[k], color=colors[k])
    
        if draw_SNR_maxima:
            ylim = 1.1*amax(hstack(data))
            for k in arange(3, 5):
                if maxima[k] < len(plot_params):
                    axes[ax_index].vlines(plot_params[maxima[k]], 0, ylim, color=colors[k%len(colors)], linestyles=':', 
                                   label='SNR maxima' if k == 3 else None)
        axes[ax_index].grid()
        #axes[2].legend(loc='upper right')
        axes[ax_index].set_ylabel('sample transparency')
        axes[ax_index].set_xlabel(xlabel)
        axes[ax_index].set_ylim(0., ylim)
        if xlog:
            axes[ax_index].set_xscale('log')
        
        
    fig.tight_layout()
    display(fig)
    plt.close(fig)
    
    if save_path is not None and saving_enabled:
        fig.savefig(save_path+'.pdf')
        print('saved plot to', save_path+'.pdf')
    return fig
    
stride = 1  # increase this value to up to 10 to speed up display at the cost of bad sampling

# 4.2 simple spectra simulation: fig. 4

higher tube voltage = higher degree of polychromaticity

In [ ]:
tube_voltages = linspace(10, 60, 151)[::stride]
tube_voltages_spectra_plot = int32(logspace(log10(tube_voltages.min()), log10(tube_voltages.max()), 5))
print(tube_voltages_spectra_plot)
plot_mask = zeros(len(tube_voltages), '?')
for tv in tube_voltages_spectra_plot:
    plot_mask[argmin(abs(tube_voltages-tv))] = True
sample_thickness = 0.1

resultsA = []
for k, E_tube in enumerate(tube_voltages):
    sim = SNR_Simulation(verbose=1)
    sim.set_source_bremsspektrum(E_tube, limit='power')
    sim.add_filter(sample_thickness/3)
    sim.add_absorbing_signal(sample_thickness*2/3)
    sim.simulate('')
    sim.plot_param = E_tube
    
    resultsA.append(sim)

print(sim.log)
print(f'monochromatic energy for (photo-)absorption: {sim.E_mono:.3f} keV')

resultsB = []
for k, E_tube in enumerate(tube_voltages):
    sim = SNR_Simulation(verbose=1)
    sim.set_source_bremsspektrum(E_tube, limit='power')
    sim.add_filter(sample_thickness/3)
    sim.add_absorbing_signal(sample_thickness*2/3, phase_only=True)
    sim.add_phase_signal(1)
    sim.simulate('')
    sim.plot_param = E_tube
    resultsB.append(sim)
print(f'monochromatic energy for phase: {sim.E_mono:.3f} keV')
    
x = plot_comparison(resultsA, figsize=(5, 6), spectra_mask=plot_mask, ls='-', markers=('', '', '', '', ''),
                 plot_transparency=0, save_path='fig/SNR_spectra_sim_example_thin', draw_SNR_maxima=True)
x = plot_comparison(resultsB, figsize=(5, 3.6), spectra_mask=plot_mask, ls='-', markers=('', '', '', '', ''), 
                    plot_sources=0, plot_transparency=0, 
                    save_path='fig/SNR_spectra_sim_example_thin_phase', draw_SNR_maxima=True)

# 4.3 screen thickness: fig. 5

In [ ]:
def plot_comparison_det(results, relative_SNR=False, save_path=None, figsize='auto', 
                    spectra_mask=None, markers=('1', '2', '2', '1', '3'), ls='', 
                    draw_SNR_maxima=True, relative_yscale=True, xlabel='tube voltage [keV]', xlog=False, closefig=True):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    plot_params = [result.plot_param for result in results]
    if spectra_mask is None:
        spectra_mask = ones(len(results), '?')
    if figsize == 'auto':
        figsize = (8, 11)
        
    fig = plt.figure(figsize=figsize)
    naxes = 3
    gs = fig.add_gridspec(naxes, 1, height_ratios=(5, 3, 3)[:naxes])
    axes = []
    for k in range(naxes):
        axes.append(fig.add_subplot(gs[k]))#, sharex=(axes[0] if k > 0 else None)))
    
    if relative_SNR:
        norm = array([r.SNR_mono for r in results])
    else:
        norm = 1.
        
    SNR_noweight = array([r.SNR_noweight for r in results])/norm
    SNR_EI_weight = array([r.SNR_EI_weight for r in results])/norm
    SNR_opt_weight = array([r.SNR_opt_weight for r in results])/norm
    SNR_lim_weight = array([r.SNR_lim_weight for r in results])/norm
    SNR_mono = array([r.SNR_mono for r in results])/norm
    all_SNR = hstack((SNR_noweight, SNR_EI_weight, SNR_opt_weight, SNR_lim_weight, SNR_mono))
    dynamic_range = all_SNR.max()/all_SNR.min()
    
    plot_kwargs = {'ls': ls, 'markersize':10}
    labels = ('monochrom.', 'optimal CEW', 'optimal DEW', 'no weight', '$\propto$E CEW')
    data = SNR_mono, SNR_opt_weight, SNR_lim_weight, SNR_noweight, SNR_EI_weight
    
    ylim = 1.1*amax(all_SNR)
    if relative_yscale:
        norm = all_SNR.max()
        ylim /= norm
        
    axes[0].plot(plot_params, data[0]/norm, marker=markers[0], label=labels[0], color=colors[0], 
                 markersize=plot_kwargs['markersize'], ls=('--' if plot_kwargs['ls'] == '-' else plot_kwargs['ls']))
    for k in arange(1, 5):
        axes[0].plot(plot_params, data[k]/norm, marker=markers[k], label=labels[k], color=colors[k], **plot_kwargs)
    if draw_SNR_maxima:
        maxima = array([argmax(d) for d in data])
        labeled = False
        for k in arange(3, 5):
            if maxima[k] < len(plot_params)-1:
                axes[0].vlines(plot_params[maxima[k]], 0, ylim, color=colors[k%len(colors)], linestyles=':', 
                               label='SNR maxima' if not labeled else None)
                labeled = True
            else:
                print('ignored SNR maxima vline for', labels[k])
    axes[0].grid()
    axes[0].set_yscale('log' if dynamic_range > 20 else 'linear')
    if xlog:
        axes[0].set_xscale('log')
    axes[0].legend(loc='upper right')
    axes[0].set_ylabel('SNR [au]')
    axes[0].set_xlabel(xlabel)
    axes[0].set_ylim(0., ylim)
        
    data = array([r.DA_mono for r in results]), array([r.DA_CEW for r in results]), array([r.DA_DEW for r in results]), array([r.DA_noweight for r in results]), array([r.DA_EI for r in results])        
    axes[1].plot(plot_params, data[0], label=labels[0], color=colors[0], 
                 ls=('--' if plot_kwargs['ls'] == '-' else plot_kwargs['ls']))
    for k in arange(1, 5):
        axes[1].plot(plot_params, data[k], label=labels[k], color=colors[k])

    if draw_SNR_maxima:
        ylim = 1.1*amax(hstack(data))
        labeled = False
        for k in arange(3, 5):
            if maxima[k] < len(plot_params):
                axes[1].vlines(plot_params[maxima[k]], 0, ylim, color=colors[k%len(colors)], linestyles=':', 
                               label='SNR maxima' if not labeled else None)
                labeled = True
                
    axes[1].grid()
    #axes[2].legend(loc='upper right')
    axes[1].set_ylabel('average absorption')
    axes[1].set_xlabel(xlabel)
    axes[1].set_ylim(0., ylim)
    if xlog:
        axes[1].set_xscale('log')
        
    
    thickness_indices = linspace(0, len(plot_params)-1, 4, dtype='i4')
    for k, k_t in enumerate(thickness_indices):
        r = results[k_t]   
        axes[2].plot(r.E, r.detector_abs_spectrum, label=f'$t$ = 10$^{log10(plot_params[k_t]):.0f}$', color=colors_B[k])

    axes[2].grid()
    axes[2].legend(loc='upper right')
    axes[2].set_ylabel('detector absorption')
    axes[2].set_xlabel('energy [keV]')
    #axes[1].set_ylim(0., ylim)
    #if xlog:
    #    axes[1].set_xscale('log')
        
        
    fig.tight_layout()
    display(fig)
    if closefig:
        plt.close(fig)
    
    if save_path is not None:
        fig.savefig(save_path+'.pdf')
    return fig
    
stride = 1  # increase this value to up to 10 to speed up computations at the cost of bad sampling

In [ ]:
sample_thickness = 0.1
E_tube = 38
screen_thicknesses = logspace(2, 5, 161)

results = []
for k, screen_thickness in enumerate(screen_thicknesses[::stride]):
    sim = SNR_Simulation(verbose=1)
    sim.set_source_bremsspektrum(E_tube, limit='power')
    sim.set_photoabs_detector(screen_thickness)
    sim.add_filter(sample_thickness/3)
    sim.add_absorbing_signal(sample_thickness*2/3)
    sim.simulate()
    sim.plot_param = screen_thickness
    
    results.append(sim)

print(sim.log)
print(f'monochromatic energy for (photo-)absorption: {sim.E_mono:.3f} keV')

x = plot_comparison_det(results, figsize=(5, 7), ls='-', markers=('', '', '', '', ''), 
                        save_path='fig/SNR_screen_thickness_example', draw_SNR_maxima=True, 
                        xlabel='$t$ [au]', xlog=True)

In [ ]:
%%time
# this simulation takes a long time, up to 1 hour
stride = 1

tube_voltages = linspace(10, 60, 51)[::stride]
sample_thickness = 0.1
opt_avg_T_noweight = zeros(len(tube_voltages), 'f8')
opt_avg_T_EI_weight = zeros(len(tube_voltages), 'f8')
screen_thicknesses = logspace(2, 5, 201)
nsims, nt = len(tube_voltages)*len(screen_thicknesses), len(screen_thicknesses)

for k, E_tube in enumerate(tube_voltages):
    sim = SNR_Simulation(verbose=1)
    sim.set_source_bremsspektrum(E_tube, limit='power')
    sim.add_filter(sample_thickness/3)
    sim.add_absorbing_signal(sample_thickness*2/3)

    avg_T_noweight = zeros(len(screen_thicknesses), 'f8')
    avg_T_EI_weight = zeros(len(screen_thicknesses), 'f8')
    SNR_noweight = zeros(len(screen_thicknesses), 'f8')
    SNR_EI_weight = zeros(len(screen_thicknesses), 'f8')
    for k_t, screen_thickness in enumerate(screen_thicknesses[::stride]):
        sim.set_photoabs_detector(screen_thickness)
        sim.simulate('d')
        
        avg_T_noweight[k_t] = sim.DA_noweight
        avg_T_EI_weight[k_t] = sim.DA_EI
        SNR_noweight[k_t] = sim.SNR_noweight
        SNR_EI_weight[k_t] = sim.SNR_EI_weight
        if (nt*k + k_t + 1) % (nsims//10) == 0:
            print(f'{(nt*k + k_t) / nsims*100:.0f} % done')
    
        
    opt_avg_T_noweight[k] = avg_T_noweight[argmax(SNR_noweight)]
    opt_avg_T_EI_weight[k] = avg_T_EI_weight[argmax(SNR_EI_weight)]

print(sim.log)    
        
fig, ax = plt.subplots(1, 1, figsize=(5, 2.5))
ax.plot(tube_voltages, opt_avg_T_noweight, color=colors[3], label='no weight')
ax.plot(tube_voltages, opt_avg_T_EI_weight, color=colors[4], label='$\propto E$ CEW')

ax.grid()
ax.legend(loc='upper right')
ax.set_ylabel('avg. abs. optimum')
ax.set_xlabel('tube voltage [kV]')
ax.set_ylim(0., 1.05)

fig.tight_layout()
display(fig)
fig.savefig('fig/SNR_screen_thickness_abs_opt.pdf')

In [ ]:
plt.show()